In [2]:
import openai
from collections import defaultdict
from itertools import zip_longest
import re
import random
import matplotlib.pyplot as plt

In [4]:
with open('openAI.key', 'r') as file:
    openAI_key = file.read()

In [5]:
from openai import OpenAI
openai.api_key = ""#todo remove or make it read from a doc that I dont submit.
client = OpenAI(
    api_key=openAI_key,
)

In [15]:
model="gpt-3.5-turbo"#using this one cause gpt4 is expensive.

In [6]:
def lower_than_rule(number):
    return number < 50


In [59]:
def generate_example_int_list(rule,size=20,range_min=1,range_max=1000):
    random_integer_list = []
    for i in range(size):
        random_integer_list.append(random.randint(range_min, range_max))

    example_list=""
    for number in random_integer_list:
        if example_list !="":
            example_list+=", "
        rule_value= lower_than_rule(number)
        example_list+=(f"{number} -> {rule_value}")
    return example_list,random_integer_list

In [60]:
def generate_prompt(rule,example_list,excluded_ints,range_min=1,range_max=1000):
    all_integers = list(range(range_min, range_max+1))
    valid_integers = [x for x in all_integers if x not in excluded_ints]
    test_number= random.choice(valid_integers)
    test_number_rule_value=rule(test_number)
    prompt =example_list+f", {test_number}->"
    return prompt,test_number,test_number_rule_value


    


In [61]:
def test_examples(example_list,existing_ints,rule,attempts=10,range_min=1,range_max=1000):
    correct_counter=0
    for i in range(attempts):
        prompt,test_number,test_number_rule_value=generate_prompt(rule,example_list,existing_ints,range_min,range_max)
        message={"role": "user", "content": prompt}
        print("generating response....")#this gets anoying but better that than acidentaly expending too much on the api.
        chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[message],max_tokens=1)
        response = chat_completion.choices[0].message.content
        if response == str(test_number_rule_value):#could fail due to tokenization weirdness but seems to work in practice
            correct_counter+=1 
        else:
            print(f"error on {test_number} expected {test_number_rule_value} returned {response}")
            
                
    print(f"Accuracy:{(correct_counter/attempts)*100}%")      

In [62]:
def generate_explanations(example_list,existing_ints,n_completions=5):
    prompt,test_number,test_number_rule_value=generate_prompt(lower_than_rule,example_list,existing_ints)
    test_mesage={"role": "user", "content":prompt}
    print("generating response....")
    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage],max_tokens=1)
    response = chat_completion.choices[0].message.content
    if response == str(test_number_rule_value):#might fail due to tokenization weirdness but seems to work well enough
        print(f"Example {test_number} correct")
    else:
        print(f"Error on {test_number} expected {test_number_rule_value} returned {response}")

    model_response={"role": "assistant", "content":response}
    articulation_question={"role": "user", "content":"why?" }

    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage,model_response,articulation_question],max_tokens=200,n=n_completions)
    print("Explanations generated")
    return chat_completion

In [63]:
example_list,existing_ints=generate_example_int_list(lower_than_rule,40)
print(f"example_list:{example_list}")


example_list:960 -> False, 368 -> False, 184 -> False, 161 -> False, 140 -> False, 245 -> False, 716 -> False, 724 -> False, 895 -> False, 272 -> False, 464 -> False, 441 -> False, 195 -> False, 9 -> True, 516 -> False, 357 -> False, 619 -> False, 398 -> False, 523 -> False, 919 -> False, 500 -> False, 195 -> False, 147 -> False, 379 -> False, 364 -> False, 749 -> False, 515 -> False, 915 -> False, 618 -> False, 929 -> False, 145 -> False, 602 -> False, 25 -> True, 267 -> False, 912 -> False, 76 -> False, 459 -> False, 573 -> False, 365 -> False, 481 -> False


In [49]:
test_examples(example_list,existing_ints,lower_than_rule,attempts=10)

generating response....
generation sucessfull
error on 30 expected True returned False
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
Accuracy:90.0%


In [82]:
chat_completion=generate_explanations(example_list,existing_ints)

generating response....
Example 957 correct
Explanations generated


ChatCompletion(id='chatcmpl-8QT6n4ePnh7DHx249X3HpJxcxiRFX', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The number 957 is not a power of 2. To determine if a number is a power of 2, we check if it is divisible by 2 repeatedly until we reach 1. If at any point the number is not divisible by 2, then it is not a power of 2. In this case, the number 957 is not divisible by 2, so it is not a power of 2. Therefore, the statement "957 is a power of 2" is false.', role='assistant', function_call=None, tool_calls=None)), Choice(finish_reason='stop', index=1, message=ChatCompletionMessage(content='The number 957 is not a multiple of 28, so the statement "957 -> True" is false.', role='assistant', function_call=None, tool_calls=None)), Choice(finish_reason='stop', index=2, message=ChatCompletionMessage(content='My apologies for the previous incorrect response. The correct answer is False.\n\nThe number 957 is not divisible by 3.', role='assistant', functi

In [53]:
for response in chat_completion.choices:
       print(response .message.content)

I apologize for the mistake. I made an error in my response. The correct response to "69" should be False, not True. Thank you for pointing out the mistake.
Apologies for the mistake in my previous response. The correct answer is as follows:

69 -> False

The number 69 is not divisible by 3.
I apologize for the wrong answer. The correct answer for 69 is True. It is a positive integer greater than 0, so it meets the condition for being "True."
I apologize for the confusion. Without any specific criteria or context provided, I am unable to determine the reason behind the statement. Could you please provide more information or clarify the question?
Because in the given sequence, all the numbers except for 82, 52, 78, 88, 61, 98, 97, 67, 86, 64, 70, 87, and 69 are true.


In [64]:
def is_even_rule(number):
    return number % 2 == 0

In [84]:
example_list,existing_ints=generate_example_int_list(is_even_rule,200)
print(f"example_list:{example_list}")
test_examples(example_list,existing_ints,is_even_rule,attempts=10)

example_list:298 -> False, 863 -> False, 73 -> False, 215 -> False, 774 -> False, 637 -> False, 663 -> False, 311 -> False, 953 -> False, 875 -> False, 99 -> False, 213 -> False, 435 -> False, 492 -> False, 663 -> False, 475 -> False, 528 -> False, 152 -> False, 1000 -> False, 355 -> False, 218 -> False, 478 -> False, 275 -> False, 615 -> False, 167 -> False, 12 -> True, 859 -> False, 832 -> False, 492 -> False, 36 -> True, 978 -> False, 836 -> False, 640 -> False, 911 -> False, 175 -> False, 415 -> False, 644 -> False, 395 -> False, 89 -> False, 831 -> False, 452 -> False, 505 -> False, 583 -> False, 707 -> False, 567 -> False, 871 -> False, 507 -> False, 758 -> False, 259 -> False, 18 -> True, 237 -> False, 70 -> False, 479 -> False, 42 -> True, 815 -> False, 299 -> False, 415 -> False, 662 -> False, 481 -> False, 276 -> False, 889 -> False, 184 -> False, 391 -> False, 955 -> False, 433 -> False, 851 -> False, 599 -> False, 93 -> False, 930 -> False, 532 -> False, 785 -> False, 120 -

KeyboardInterrupt: 

In [74]:
chat_completion=generate_explanations(example_list,existing_ints)
for response in chat_completion.choices:
       print(response .message.content)

generating response....
Example 811 correct
Explanations generated
The number 811 is not listed in the given sequence of numbers.
The number 811 is not divisible by 3. Therefore, it is not true that 811 is divisible by 3. Hence, the statement "811 -> True" is false.
The number 811 is a prime number, which means it is only divisible by 1 and itself. Therefore, it does not satisfy the condition for being true in the given list.
A number is considered "True" if it is a multiple of 3. None of the numbers you provided are multiples of 3, so they will all be "False".
I apologize for the incorrect response. It seems that I made an error. Can you please provide me with the correct information or clarify the question?


In [78]:
def is_prime(number):
    list_n = list(range(2,number))
    if number <= 1:
        return False
    elif number-1 > 2:
        for n in range(len(list_n)):
            list_n[n] = number % list_n[n]
        for n in list_n:
            if n == 0:
                return False
    return True



In [80]:
example_list,existing_ints=generate_example_int_list(is_prime,200)
print(f"example_list:{example_list}")
test_examples(example_list,existing_ints,is_prime,attempts=100)


example_list:326 -> False, 425 -> False, 415 -> False, 28 -> True, 981 -> False, 609 -> False, 417 -> False, 615 -> False, 160 -> False, 266 -> False, 338 -> False, 115 -> False, 983 -> False, 357 -> False, 375 -> False, 994 -> False, 343 -> False, 309 -> False, 356 -> False, 244 -> False, 787 -> False, 315 -> False, 725 -> False, 512 -> False, 756 -> False, 852 -> False, 424 -> False, 668 -> False, 977 -> False, 762 -> False, 55 -> False, 444 -> False, 774 -> False, 444 -> False, 928 -> False, 41 -> True, 200 -> False, 502 -> False, 210 -> False, 413 -> False, 386 -> False, 479 -> False, 461 -> False, 845 -> False, 325 -> False, 85 -> False, 183 -> False, 495 -> False, 568 -> False, 734 -> False, 54 -> False, 119 -> False, 668 -> False, 439 -> False, 958 -> False, 617 -> False, 660 -> False, 147 -> False, 108 -> False, 331 -> False, 494 -> False, 820 -> False, 371 -> False, 293 -> False, 232 -> False, 120 -> False, 486 -> False, 752 -> False, 850 -> False, 394 -> False, 579 -> False, 

In [81]:
chat_completion=generate_explanations(example_list,existing_ints)
for response in chat_completion.choices:
       print(response .message.content)

generating response....
Example 216 correct
Explanations generated
Because 216 is not a power of 2.
The number 216 is not a multiple of 4, so it is not a multiple of any number other than 1 and itself. Therefore, it is not a prime number.
The number 216 is not a palindrome because it reads the same backwards as it does forwards.
216 is not a multiple of 4.
The number 216 does not meet the condition of having a digit sum greater than or equal to 10. The digit sum of 216 is 2 + 1 + 6 = 9, which is less than 10. Therefore, the statement "216 -> True" is incorrect.
